In [6]:
import requests 
import urllib
import pandas as pd

PAGE_SIZE = 500

places = {
    'Metzingen':11185,
    'Estorf': 11001,
    'Leese': 10746,
    'Lüdingen':9759,
    'Brunsberg': 9844,
    'Kella': 9521,
    'Börry': 9403,
    'Porta': 9712,    
    'Königszinne': 11489,
    'Rammelsberg': 9427    
}

results = []
for place in ['Metzingen']:
    limit = PAGE_SIZE    
    for start in [0, PAGE_SIZE]:
        query = {"navpars":{"start":start,"limit":limit,"sort":[{"field":"FlightDate","dir":-1},{"field":"BestTaskPoints","dir":-1}]}}
        # for some reason, " is replaced by ' and spaces are added which is replaced by +
        decoded_url = f"https://de.dhv-xc.de/api/fli/flights?s?fkcat%5B%5D=1&fkto%5B%5D={places[place]}&{urllib.parse.urlencode(query,quote_via=urllib.parse.quote_plus).replace('%27', '%22').replace('+', '')}"
        #print(decoded_url)
        r = requests.get(decoded_url)
        if r.status_code==200:
            response = r.json()
            df = pd.DataFrame(response['data'])
            #print(df.columns.values)
            results.extend(df[['FlightDate', 'TakeoffWaypointName' , 'Glider' ,'FlightDuration', 'BestTaskType','BestTaskDistance','BestTaskPoints']].values)


df_results = pd.DataFrame(results,columns=['FlightDate', 'Takeoff' , 'Glider' ,'Duration', 'Type','Distance','Points'])
print(len(df_results))
df_results.tail()

1000


,FlightDate,Takeoff,Glider,Duration,Type,Distance,Points
995,2018-05-26,Metzingen,Anakis3 - XL [LTF 1],448,Freie Strecke,2487,3.7305
996,2018-05-26,Metzingen,Mentor 4 M,327,Flaches Dreieck,2108,3.689
997,2018-05-26,Metzingen,Mentor 4 M,471,Freie Strecke,2322,3.483
998,2018-05-26,Metzingen,Anakis3 - XL [LTF 1],372,Flaches Dreieck,1914,3.3495
999,2018-05-26,Metzingen,Anakis3 - XL [LTF 1],253,Flaches Dreieck,1860,3.255


In [21]:
df_results['Year'] = df_results.apply(lambda row: row['FlightDate'][:4], axis=1)

df_50pts = df_results[df_results['Points'].astype(float)>=50.0][['Points','Year']].groupby('Year').aggregate(['count']).sort_index(ascending=False)

print(df_50pts.to_markdown())

df_30pts = df_results[df_results['Points'].astype(float)>=30.0][['Points','Year']].groupby('Year').aggregate(['count']).sort_index(ascending=False)

print(df_30pts.to_markdown())

|   Year |   ('Points', 'count') |
|-------:|----------------------:|
|   2024 |                     4 |
|   2023 |                    11 |
|   2022 |                     7 |
|   2021 |                     1 |
|   2020 |                     3 |
|   2019 |                     6 |
|   2018 |                    14 |
|   Year |   ('Points', 'count') |
|-------:|----------------------:|
|   2024 |                    10 |
|   2023 |                    23 |
|   2022 |                    11 |
|   2021 |                     3 |
|   2020 |                     8 |
|   2019 |                     9 |
|   2018 |                    20 |
